<a href="https://colab.research.google.com/github/leemordechai/FLAME/blob/master/CHRE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# the 3 required files are in Dropbox; hoards and coins were downloaded from CHRE, rulers comes from FLAME [orig. an excel file I received]
# if on Google Colab
from google.colab import files
import pandas as pd
import datetime as dt
import io
import ast

hoards_file = files.upload()

Saving hoard_export_2018_11_19_171440.csv to hoard_export_2018_11_19_171440.csv


In [2]:
coins_file = files.upload()

Saving coins_export_2018_11_19_171846.csv to coins_export_2018_11_19_171846.csv


In [15]:
rulers_file = files.upload()

Saving rulers.tsv to rulers.tsv


In [0]:
hoard_coins = pd.read_csv('coins_export_2018_11_19_171846.csv')
hoards = pd.read_csv('hoard_export_2018_11_19_171440.csv')

In [0]:
# merges the comments in CHRE system to one string
def merge_comments(first, second, third, fourth, fifth):
  combined = []
  for i in range(len(first)):
    temp = ''
    if type(first[i]) == str: temp += first[i]
    if type(second[i]) == str: temp += '//  Find spot comment: ' + second[i]
    if type(third[i]) == str: temp += '//  Discovery comment: ' + third[i]
    if type(fourth[i]) == str: temp += '//  Archaeology site comment: ' + fourth[i]
    if type(fifth[i]) == str: temp += '//  Rating comment: ' + fifth[i]
    combined.append(temp)
  return combined

# merges references in CHRE system to one string
def merge_references(first, second):
  combined = []
  for i in range(len(first)):
    temp = ''
    if type(first[i]) == str: temp += first[i]
    if type(second[i]) == str: temp += '//  ' + second[i]
    combined.append(temp)
  return combined


# function to build the coin_finds dataframe
def setting_coin_finds(hoards):
  cols_finds = ['hoard_id', 'name', 'startDate', 'endDate', 'type_find', 'hoard?', 'excavation?', 'single?', 'num_coins', 'num_known_coins', 'num_unknown_coins', 'year_found',
    'year_end_found', 'comments', 'lat', 'long', 'certainty', 'references', 'owner', 'created', 'imported']
  coin_finds = pd.DataFrame(columns=cols_finds)

  #print(coin_finds.info)
  coin_finds['hoard_id'] = hoards['id'].apply(lambda x: 'CHRE-' + str(x))
  coin_finds['name'] = hoards['findSpotName']
  coin_finds['startDate'] = hoards[['openingYear1', 'openingYear2', 'terminalYear1']].min(axis=1)
  coin_finds['endDate'] = hoards[['openingYear2', 'terminalYear1', 'terminalYear2']].max(axis=1)
  coin_finds['hoard?'] = 1
  coin_finds['excavation?'] = 0
  coin_finds['single?'] = 0
  coin_finds['type_find'] = 'hoard'
  coin_finds['num_coins'] = hoards['coinCount']
  #coin_finds['num_known_coins'] = ths['Number']
  #coin_finds['num_unknown_coins'] = ths['Number']
  coin_finds['lat'] = hoards['lattitude']
  coin_finds['long'] = hoards['longitude']
  coin_finds['owner'] = 'CHRE'
  coin_finds['year_found'] = hoards['discoveryYear1']
  coin_finds['year_end_found'] = hoards[['discoveryYear1', 'discoveryYear2']].max(axis=1)
  coin_finds['created'] = pd.Timestamp.now()
  coin_finds['imported'] = pd.Timestamp.now()
  coin_finds['comments'] = merge_comments(hoards['comment'], hoards['findSpotComment'], 
                           hoards['discoveryComment'], hoards['discoveryComment'], hoards['ratingComment'])
  coin_finds['references'] = merge_references(hoards['reference_string'], hoards['references'])
  
  return coin_finds

In [0]:
finds = setting_coin_finds(hoards)

In [0]:
# function to set the coin_groups dataframe
def setting_coin_groups(coins):
  cols = ['hoard_id', 'coin_group_id', 'start_year', 'end_year', 'revised_start', 'revised_end', 'ruler', 'revised_ruler',
    'denomination', 'num_coins', 'mint', 'imported', 'created', 'updated', 'comments']
  coin_groups = pd.DataFrame(columns=cols)
  
  coin_groups['hoard_id'] = coins['hoard'].apply(lambda x: 'CHRE-' + str(x))
  coin_groups['coin_group_id'] = coins['id'].apply(lambda x: 'CHRE-' + str(x))
  coin_groups['start_year'] = coins['startingDate']
  coin_groups['end_year'] = coins['endingDate']
  coin_groups['revised_start'] = coin_groups['start_year']
  coin_groups['revised_end'] = coin_groups['end_year']
  coin_groups['ruler'] = coins['reign']
  coin_groups['revised_ruler'] = coin_groups['ruler']
  coin_groups['metal'] = coins['material']
  coin_groups['denomination'] = coins['denomination']
  coin_groups['num_coins'] = coins['quantity']
  coin_groups['mint'] = coins['mint']
  coin_groups['imported'] = pd.Timestamp.now()	# this and the next two lines are identical because importing should be a one-off thing.
  coin_groups['created'] = pd.Timestamp.now()
  coin_groups['updated'] = pd.Timestamp.now()
  
  return coin_groups

coin_groups_org = setting_coin_groups(hoard_coins)

In [0]:
# cell used for converting CHRE-style denomination to FLAME style
denomination_conversion = {'AE (brass or copper)':'uncertain (bronze)', 'AR (silver)':'uncertain (silver)',
                            'AV (gold)':'uncertain (gold)', 'Antoninianus (radiate) OR Nummus':'radiate or nummus (UK find)',
                            'Aureus':'aureus', 'Drachm':'drachm',
 'Half solidus':'semissis', # should be the same
 'Medallion (AV)': 'medallion (gold)',
 'Miliarensis': 'miliarensis',
 'Multiple of 1 1/2 solidus': '1.5 solidus', # new demonination
 'Multiple of 1 1/4 solidus': '1.25 solidus', # new demonination
 'Multiple of 18 solidi': '18 solidi', # new demonination
 'Multiple of 2 solidi': '2 solidi', # new demonination
 'Multiple of 3 solidi': '3 solidi',
 'Multiple of 4 1/2 solidi': '4.5 solidi',
 'Nummus': '1 nummus',
 'Nummus OR Antoninianus (radiate)': 'radiate or nummus (UK find)',
 'Semis':'semissis',
 'Siliqua':'siliqua',
 'Solidus':'solidus',
 'Solidus (multiple of)':'medallion (gold)',
 'Tremissis':'tremissis',
 'Uncertain': 'uncertain',
 'Uncertain (AR)':'uncertain (silver)'}
                            
                            
obsolete_denominations = ['Antoninianus (radiate)','Antoninianus (radiate) OR Nummus', 'Argenteus', 'As', 'As OR Dupondius', 'Denarius', 'Denarius OR Antoninianus (radiate)', 'Dupondius',
                          'Prutah', 'Quadrans', 'Quinarius', 'Radiate fraction', 'Sestertius', 'Tetradrachm OR Uncertain']

In [0]:
# cell used to set CHRE mints to FLAME mints
mint_conversion = {'Trier (Treveri)':'Colonia Augusta Treverorum', 'Milan (Mediolanum)':'Mediolanum', 
                  'Rome':'Roma', 'Lyon (Lugdunum)':'Lugdunensium', 'Arles (Arelate)':'Arelato',
                  'Italic mint':'Unknown (Italy)', 'Gaul':'Unknown (Gaul)', 'Gallic mint':'Unknown (Gaul)',
                  'Cyzicus (Mysia)':'Kyzikos', 'Thessalonica (Macedonia)':'Thessalonika', 
                  'Nicomedia (Bithynia)':'Nikomedia', 'Eastern imperial mint':'Unknown (East Roman)',
                  'London (Londinium)': 'Londinium', 'Rome OR Uncertain':'Roma', 
                  'Lyon (Lugdunum) OR Vienna (Gaul)': 'Lugdunum or Vienne', 'Tripolis (Phoenicia)':'Tripoli',
                  'Cologne':'Colonia', 'Balkan mint':'Unknown (Balkan)', 'Ravenna':'Ravenna', 'Antioch (Syria)':'Antioch',
                  'Aquileia':'Aquileia', 'Provincia Dacia':'Unknown (Dacia)', 'Asia':'Unknown (Asia Minor)',
                  'Antioch (Syria) OR Uncertain':'Antioch', 'Amiens (Ambianum)':'Ambianum', 'Carthage':'Carthago', 
                   'Sicily':'Sicily',
                  'Siscia OR Thessalonica (Macedonia)':'Siscia or Thessalonika', # new mint
                  'Aquileia OR Trier (Treveri)':'Aquileia or Trier', # new mint
                  'Aquileia OR Thessalonica (Macedonia)':'Aquileia or Thessalonika', # new mint
                  'Alexandria OR Uncertain':'Alexandria ad Aegyptum',
                  'Uncertain':'Unknown',
                  'Jerusalem (Judaea)':'Jerusalem',
                  'Local':'Unknown',
                  'Gallic mint OR Rome':'Gallic mint or Roma', # new mint
                  'Ravenna OR Rome':'Ravenna or Roma', # new mint
                  'Arles (Arelate) OR Constantinople':'Arelato or Constantinople', # new mint
                  'Arles (Arelate) OR Trier (Treveri)':'Arelato or Colonia Augusta Treverorum', # new mint
                  'Nicomedia (Bithynia) OR Uncertain':'Nikomedia',
                  'Heraclea OR Nicomedia (Bithynia)':'Heraclea or Nikomedia', # new mint
                  'Alexandria':'Alexandria ad Aegyptum',
                  'Aquileia OR Rome':'Aquileia or Roma', # new mint
                  'Lyon (Lugdunum) OR Trier (Treveri)':'Lugdunensium or Colonia Augusta Treverorum', # new mint
                  'Arles (Arelate) OR Lyon (Lugdunum)':'Arelato or Lugdunensium', # new mint
                  'Milan (Mediolanum) OR Ravenna':'Mediolanum or Ravenna', # new mint
                  'Travelling mint':'Unknown',
                  'Constantinople':'Constantinople',
                  'Sirmium':'Sirmium',
                  'Heraclea':'Heraclea',
                  'Siscia':'Siscia',
                  'Ticinum':'Ticinum'
                  }

# 20.11 check Ambianis & Ambianum mints

obsolete_mints = {'Viminacium', 'Marcianopolis (Moesia Inferior)', 'Tyre (Phoenicia)',
                 'Caesarea ad Libanum (Phoenicia)', 'Petra (Arabia)', 'Uncertain OR Tyre (Phoenicia)',
                 'Nicopolis ad Istrum (Moesia Inferior)', 'Serdica (Thrace)', 'Tarraco', 'Caesarea Paneas (Syria)',
                 'Nemausus', 'Philippopolis (Arabia)', 'Mesembria (Thrace)', 'Tomis (Moesia Inferior)',
                 'Uncertain OR Sebaste (Samaria)', 'Attaleia (Pampylia)', 'Tenedos (Troas)', 
                 'Uncertain OR Caesarea Paneas (Syria)','Apollonia (Illyria)', 'Ptolemais Acco (Phoenicia)',
                 'Dyrrhachium', 'Alexandria Troas', 'Heliopolis (Syria)', 'Anchialus (Thrace)', 
                 'Hadrianopolis (Thrace)', 'Callatis (Moesia Inferior)', 'Stobi (Macedonia)', 'Nicaea (Bithynia)',
                 'Sidon (Phoenicia)',
                 }

In [0]:
# actual conversion of denominations and mints to FLAME style

coin_groups = coin_groups_org[~coin_groups_org['denomination'].isin(obsolete_denominations)]
new_list = coin_groups['denomination'].fillna('Uncertain').apply(lambda x:denomination_conversion[x])
coin_groups['denomination'] = new_list
#coin_groups.head() # at this point the denominations are okay

coin_groups = coin_groups[~coin_groups['mint'].isin(obsolete_mints)]
new_list = coin_groups['mint'].fillna('Uncertain').apply(lambda x:mint_conversion[x])
coin_groups['mint'] = new_list
#coin_groups.head() # at this point the mints are okay

In [87]:
# setting up the ruler conversion between CHRE and FLAME
set(coin_groups['ruler'])

new_ruler_list = coin_groups['ruler'].fillna('Unknown')
coin_groups['ruler'] = new_ruler_list

irrelevant_rulers = ['Antoninus Pius', 'Augustus', 'Augustus OR Tiberius', 'Aurelian', 'Caracalla',
 'Carinus', 'Claudius I', 'Claudius II', 'Commodus', 'Diocletian (RIC V)', 'Diocletian (RIC V) OR Tetrarchy (RIC VI)',
 'Domitian', 'Elagabalus', 'Gaius', 'Gallienus (sole)', 'Gordian III', 'Hadrian', 'Hadrian OR Uncertain', 'Licinius I',
 'Licinius I OR Tetrarchy (RIC VI)', 'Macrinus', 'Marcus Aurelius',  'Maxentius', 'Maximian (RIC V)', 
 'Maximian (RIC V) OR Tetrarchy (RIC VI)', 'Maximinus I Thrax', 'Nero', 'Philip I', 'Postumus', 'Probus',
 'Quintillus', 'Septimius Severus',  'Severus Alexander', 'Tetrarchy (RIC VI)', 'Tetricus I', 'Tiberius',
 'Trajan', 'Trajan Decius', 'Trebonianus Gallus',  'Valerian I and Gallienus', 'Valerian I and Gallienus OR Gallienus (sole)',
 'Vespasian', 'Victorinus',]

ruler_list = {
    'Anastasius':('Anastasios I', 491, 518),
    'House of Constantine (RIC VIII) OR Constantine I':('House of Constantine', 306, 364),
    'House of Constantine (RIC VIII)':('House of Constantine (post Constantine)', 337, 364),
    'Constantine I OR House of Constantine (RIC VIII)':('House of Constantine', 306, 364),
    'Uncertain':('Unknown', None, None),
    'House of Valentinian':('House of Valentinian', 364, 383),
    'Zeno OR Leo I':('Zeno or Leo I', 457, 491),
    'Theodosius I OR Theodosius II':('Theodosius I or Theodosius II', 379, 450),
    'Valentinian I OR Valens OR Valentinian II OR Theodosius I OR Gratian':('Valentinian I to Theodosius I', 364, 395),
    'Valentinian I OR Valens OR Valentinian II OR Gratian':('Valentinian I to Gratian', 364, 383),
    'Constantius II OR Constans':('Constantius II or Constans', 337, 361),
    'Julian II OR Constantius II':('Julian II or Constantius II', 337, 363),
    'Valentinian II OR Theodosius II':('Valentinian II or Theodosius II', 375, 450),
    'Theodosius I OR Arcadius (RIC X)':('Theodosius I or Arcadius', 379, 408),
    'House of Valentinian OR House of Theodosius':('House of Valentinian or House of Theodosius', 364, 450),
    'House of Constantine (RIC VIII) OR House of Valentinian':('House of Constantine or House of Valentinian', 337, 383),
    'Constantine I OR Constans':('Constantine I or Constans', 306, 350),
    'Priscus Attalus':('Priscus Attalus', 409, 415),
    'Honorius (RIC X) OR Theodosius I':('Honorius or Theodosius I', 379, 423),
    'Constans':('Constans I', 337, 350),
    'Valentinian II OR Theodosius I OR Arcadius (RIC X) OR Honorius (RIC X)':('Valentinian II to Honorius', 375, 423),
    'Valentinian II OR Theodosius I':('Valentinian II or Theodosius I', 375, 395),
    'Arcadius (RIC X) OR Theodosius I':('Arcadius or Theodosius I', 395, 408),
    'Arcadius (RIC X)':('Arcadius', 395, 408),
    'Julian II OR House of Constantine (RIC VIII)':('House of Constantine (post Constantine)', 337, 364),
    'Constantius II OR Julian II':('Constantius II OR Julian II', 337, 363),
    'Constantine I OR Constantius II':('Constantine I OR Constantius II', 306, 361),
    'Heraclius (Augustus)':('Heraklios', 610, 641),
    'Julian I':('Julian', 361, 363),
    'Julian II':('Julian', 361, 363),
    'Procopius':('Procopius', 363, 366),
    'Theodosius I OR Uncertain':('Theodosius I', 379, 395),
    'Theodosius I OR Honorius (RIC X)':('Theodosius I or Honorius', 379, 423),
    'Julian II OR Uncertain':('Julian', 361, 363),
    'Constantine I OR Tetrarchy (RIC VI)':('Constantine I or Tetrarchy', 284, 337),
    'Valentinian I OR Valens':('Valentinian I OR Valens', 364, 378),
    'Honorius (RIC X) OR Theodosius II':('Honorius or Theodosius II', 395, 450),
    'Flavius Victor':('Flavius Victor', 383, 388),
    'Magnus Maximus OR Flavius Victor':('Magnus Maximus or Flavius Victor', 383, 388),
    'Valentinian III OR Theodosius II':('Valentinian III or Theodosius II', 402, 455),
    'Constans II (Augustus)':('Constans II (Western)', 407, 411),
    'Theodosius II OR Leo I':('Theodosius II or Leo I', 402, 474),
    'Constantine I OR Constantine II':('Constantine I or Constantine II', 306, 340),
    'Constantine II OR Constantine I':('Constantine I or Constantine II', 306, 340),
    'Valentinian II OR House of Theodosius':('Valentinian II or House of Theodosius', 375, 450),
    'Honorius (RIC X)':('Honorius', 395, 423),
    'House of Theodosius':('House of Theodosius', 379, 450),
    'House of Theodosius OR Valentinian II':('Valentinian II or House of Theodosius', 375, 450),
    'Visigoths':('Unknown (Visigoth)', 438, 710)
}


coin_groups_third_filter = coin_groups[~coin_groups['ruler'].isin(irrelevant_rulers)]
print(len(coin_groups_third_filter))
coin_groups_third_filter.head()

9621


,hoard_id,coin_group_id,start_year,end_year,revised_start,revised_end,ruler,revised_ruler,denomination,num_coins,mint,imported,created,updated,comments,metal
0,CHRE-588,CHRE-239304,395.0,402.0,395.0,402.0,Arcadius (RIC X),Arcadius (RIC X),solidus,1,Mediolanum,2018-12-06 19:13:53.696538,2018-12-06 19:13:53.698233,2018-12-06 19:13:53.699556,NaN,Gold
1,CHRE-588,CHRE-239306,395.0,402.0,395.0,402.0,Arcadius (RIC X),Arcadius (RIC X),solidus,1,Mediolanum,2018-12-06 19:13:53.696538,2018-12-06 19:13:53.698233,2018-12-06 19:13:53.699556,NaN,Gold
2,CHRE-588,CHRE-239307,395.0,402.0,395.0,402.0,Arcadius (RIC X),Arcadius (RIC X),solidus,1,Mediolanum,2018-12-06 19:13:53.696538,2018-12-06 19:13:53.698233,2018-12-06 19:13:53.699556,NaN,Gold
3,CHRE-588,CHRE-239305,395.0,402.0,395.0,402.0,Arcadius (RIC X),Arcadius (RIC X),solidus,1,Mediolanum,2018-12-06 19:13:53.696538,2018-12-06 19:13:53.698233,2018-12-06 19:13:53.699556,NaN,Gold
4,CHRE-588,CHRE-239316,402.0,406.0,402.0,406.0,Arcadius (RIC X),Arcadius (RIC X),solidus,1,Ravenna,2018-12-06 19:13:53.696538,2018-12-06 19:13:53.698233,2018-12-06 19:13:53.699556,NaN,Gold


In [88]:
#import from rulers file
FLAME_rulers = pd.read_csv('rulers.tsv', sep='\t', header=None)
FLAME_rulers.head()

,0,1,2
0,RulerName,RulerStartYear,RulerEndYear
1,Bonifacius,422,429
2,Gildo,397,398
3,Heraclianus,413,413
4,Godomar,523,532


In [89]:
# generate dataframe of rulers & their dates
def ruler_dates(ruler_name):
  if type(ruler_name) != str: 
    return ('Unknown', None, None)
  try:
    return ruler_list[ruler_name]
  except:
    temp = FLAME_rulers[FLAME_rulers[0] == ruler_name]
    try:
      result = (temp[0].iloc[0], int(temp[1]), int(temp[2]))
      return result
    except:
      print('error: ' + ruler_name)

ruler_times = coin_groups_third_filter['revised_ruler'].apply(lambda x: ruler_dates(x))
ruler_times.head(2)

0    (Arcadius, 395, 408)
1    (Arcadius, 395, 408)
Name: revised_ruler, dtype: object

In [0]:
import warnings
warnings.filterwarnings('ignore')

names = []
starting_dates = []
end_dates = []
for i in list(ruler_times):
  names.append(i[0])
  starting_dates.append(i[1])
  end_dates.append(i[2])

coin_groups_third_filter['revised_ruler'] = names
coin_groups_third_filter['calculated_start'] = starting_dates
coin_groups_third_filter['calculated_end'] = end_dates

coin_groups_third_filter['revised_start'] = coin_groups_third_filter['revised_start'].fillna(coin_groups_third_filter['calculated_start'])
coin_groups_third_filter['revised_end'] = coin_groups_third_filter['revised_end'].fillna(coin_groups_third_filter['calculated_end'])
coin_groups_third_filter = coin_groups_third_filter.drop(['calculated_start', 'calculated_end'], axis=1)

In [91]:
# remove all coin groups whose end date is before 325 (=too early)
coin_groups_fourth_filter = coin_groups_third_filter[(coin_groups_third_filter['revised_end'] > 325) | (coin_groups_third_filter['revised_end'].isna())]
coin_groups_fourth_filter.head(2)

,hoard_id,coin_group_id,start_year,end_year,revised_start,revised_end,ruler,revised_ruler,denomination,num_coins,mint,imported,created,updated,comments,metal
0,CHRE-588,CHRE-239304,395.0,402.0,395.0,402.0,Arcadius (RIC X),Arcadius,solidus,1,Mediolanum,2018-12-06 19:13:53.696538,2018-12-06 19:13:53.698233,2018-12-06 19:13:53.699556,NaN,Gold
1,CHRE-588,CHRE-239306,395.0,402.0,395.0,402.0,Arcadius (RIC X),Arcadius,solidus,1,Mediolanum,2018-12-06 19:13:53.696538,2018-12-06 19:13:53.698233,2018-12-06 19:13:53.699556,NaN,Gold


In [92]:
# converting the metals to FLAME's standardized format
metal_conversion = {'Brass':'brass',
                   'Copper':'copper',
                   'Copper alloy':'copper',
                   'Copper alloy OR Base Silver':'silver',
                   'Gold':'gold',
                   'Silver':'silver',
                   'Unknown':'unknown'}
coin_groups_metal_fix = coin_groups_fourth_filter['metal'].fillna('Unknown')
coin_groups_metal_fix = coin_groups_metal_fix.apply(lambda x:metal_conversion[x])
coin_groups_fourth_filter['metal'] = coin_groups_metal_fix
coin_groups_fourth_filter.head(2)

,hoard_id,coin_group_id,start_year,end_year,revised_start,revised_end,ruler,revised_ruler,denomination,num_coins,mint,imported,created,updated,comments,metal
0,CHRE-588,CHRE-239304,395.0,402.0,395.0,402.0,Arcadius (RIC X),Arcadius,solidus,1,Mediolanum,2018-12-06 19:13:53.696538,2018-12-06 19:13:53.698233,2018-12-06 19:13:53.699556,NaN,gold
1,CHRE-588,CHRE-239306,395.0,402.0,395.0,402.0,Arcadius (RIC X),Arcadius,solidus,1,Mediolanum,2018-12-06 19:13:53.696538,2018-12-06 19:13:53.698233,2018-12-06 19:13:53.699556,NaN,gold


In [93]:
# search for coins with uncertain mints
# the current solution for these is to ascribe half the number of coins to each mint
# and display only connections with mints that have >= 1 coins to them.
coin_groups_fourth_filter['num_coins'].sum()
len(coin_groups_fourth_filter) # 9575 coin groups
len(finds) # 768 hoards
problems = coin_groups_fourth_filter[coin_groups_fourth_filter['mint'].str.contains(' or ')]
problems.head()


,hoard_id,coin_group_id,start_year,end_year,revised_start,revised_end,ruler,revised_ruler,denomination,num_coins,mint,imported,created,updated,comments,metal
1065,CHRE-4469,CHRE-260762,NaN,NaN,NaN,NaN,Uncertain,Unknown,1 nummus,1,Aquileia or Roma,2018-12-06 19:13:53.696538,2018-12-06 19:13:53.698233,2018-12-06 19:13:53.699556,NaN,copper
3079,CHRE-4886,CHRE-263567,NaN,NaN,337.0,361.0,Constantius II,Constantius II,1 nummus,2,Siscia or Thessalonika,2018-12-06 19:13:53.696538,2018-12-06 19:13:53.698233,2018-12-06 19:13:53.699556,NaN,copper
3541,CHRE-7019,CHRE-178305,NaN,NaN,379.0,395.0,Theodosius I,Theodosius I,1 nummus,1,Arelato or Lugdunensium,2018-12-06 19:13:53.696538,2018-12-06 19:13:53.698233,2018-12-06 19:13:53.699556,NaN,copper
3542,CHRE-7019,CHRE-255905,NaN,NaN,379.0,395.0,Theodosius I,Theodosius I,1 nummus,1,Aquileia or Roma,2018-12-06 19:13:53.696538,2018-12-06 19:13:53.698233,2018-12-06 19:13:53.699556,NaN,copper
3743,CHRE-7354,CHRE-181769,NaN,NaN,NaN,NaN,Unknown,Unknown,uncertain (bronze),1,Lugdunum or Vienne,2018-12-06 19:13:53.696538,2018-12-06 19:13:53.698233,2018-12-06 19:13:53.699556,NaN,brass


In [94]:
# this cell splits all the mints which have two options into two separate entries
def split_entry(entry):
  solved_problem = pd.DataFrame(columns=problems.columns)
  n_coins = entry.num_coins
  c_g_id = entry.coin_group_id
  temp = entry.mint.split()
  entry.comments = 'half the coin group of an uncertain mint'
  
  #resolve first part
  entry.mint = temp[0]
  entry.num_coins = entry.num_coins / 2    # need this only once
  entry.coin_group_id = c_g_id + '-a'
  solved_problem = solved_problem.append(entry)
  
  # resolve second part
  entry.mint = temp[2]
  entry.coin_group_id = c_g_id + '-b'
  solved_problem = solved_problem.append(entry)
  return solved_problem

solved_problems = pd.DataFrame(columns=problems.columns)
for i in range(len(problems)):  
  solved_problems = solved_problems.append(split_entry(problems.iloc[i]))

solved_problems.head(2)
  

,hoard_id,coin_group_id,start_year,end_year,revised_start,revised_end,ruler,revised_ruler,denomination,num_coins,mint,imported,created,updated,comments,metal
1065,CHRE-4469,CHRE-260762-a,NaN,NaN,NaN,NaN,Uncertain,Unknown,1 nummus,0.5,Aquileia,2018-12-06 19:13:53.696538,2018-12-06 19:13:53.698233,2018-12-06 19:13:53.699556,half the coin group of an uncertain mint,copper
1065,CHRE-4469,CHRE-260762-b,NaN,NaN,NaN,NaN,Uncertain,Unknown,1 nummus,0.5,Roma,2018-12-06 19:13:53.696538,2018-12-06 19:13:53.698233,2018-12-06 19:13:53.699556,half the coin group of an uncertain mint,copper


In [96]:
print(list(problems.index.values))
# removing the old entries and appending the new entries
coin_groups_fifth_filter = coin_groups_fourth_filter.drop(list(problems.index.values))
coin_groups_fifth_filter = coin_groups_fifth_filter.append(solved_problems)

# rechecking
#problems = coin_groups_fifth_filter[coin_groups_fifth_filter['mint'].str.contains(' or ')]
#problems # should be empty: works
#coin_groups_fifth_filter.loc[6238] # should have two results: works


[1065, 3079, 3541, 3542, 3743, 3824, 5028, 6237, 6238, 6292, 6324, 6672, 7522, 8254, 8767, 9304, 9327, 9605, 10347]


,hoard_id,coin_group_id,start_year,end_year,revised_start,revised_end,ruler,revised_ruler,denomination,num_coins,mint,imported,created,updated,comments,metal
6238,CHRE-7835,CHRE-190714-a,NaN,NaN,375.0,450.0,House of Theodosius OR Valentinian II,Valentinian II or House of Theodosius,1 nummus,0.5,Arelato,2018-12-06 19:13:53.696538,2018-12-06 19:13:53.698233,2018-12-06 19:13:53.699556,half the coin group of an uncertain mint,copper
6238,CHRE-7835,CHRE-190714-b,NaN,NaN,375.0,450.0,House of Theodosius OR Valentinian II,Valentinian II or House of Theodosius,1 nummus,0.5,Constantinople,2018-12-06 19:13:53.696538,2018-12-06 19:13:53.698233,2018-12-06 19:13:53.699556,half the coin group of an uncertain mint,copper


In [23]:
# these 22 hoards have no associated coins with them - see comments below for more details. I decided to include them in FLAME despite the partial data. 
all_finds = set(finds['hoard_id'])
all_coins = set(coin_groups_fourth_filter['hoard_id'])
problem_hoards = list(all_finds.difference(all_coins))
finds[finds['hoard_id'].isin(problem_hoards)]['comments'].apply(lambda x: print(x))
finds[finds['hoard_id'].isin(problem_hoards)].head()

Hoard of 281 antoniniani (radiates) found on the territory of the legionary fort: Valerian I and Gallienus - Diocletian (AD 285).//  Find spot comment: On the territory of the legionary fort.
One silver bowl with niello cross. Four stamps of Zeno and Anastasius. Dating based on style of portraiture (AD 491-518).
The hoard comprised approximately 100 Roman silver coins in a ceramic vessel. The vessel was found inside an iron box. The coins were Roman in date. Lapierre purchased 4 of them. Three depicted Roman emperors and the last a Roman empress.//  Find spot comment: The hoard was found in Vernay.//  Discovery comment: The hoard was found by a mason while digging a well.//  Archaeology site comment: The hoard was found by a mason while digging a well.
25 coins found during peat digging, all from the reign of Constantine I.: Constantinus I. (1), Licinius I. (1), Crispus Caesar (5) and Constantine II. Caesar (8). No further indication. Interpreted as a hoard or a purse, based on the coi

,hoard_id,name,startDate,endDate,type_find,hoard?,excavation?,single?,num_coins,num_known_coins,...,year_found,year_end_found,comments,lat,long,certainty,references,owner,created,imported
8,CHRE-4256,LAURIACUM 1906 2,335.0,354.0,hoard,1,0,0,317.0,NaN,...,1906.0,1907.0,Hoard of 281 antoniniani (radiates) found on t...,48.221,14.475,NaN,"// Dembski, G. - 1977 - Die antiken Münzschat...",CHRE,2018-12-06 18:12:14.258392,2018-12-06 18:12:14.259510
59,CHRE-4690,SVETLEN 1,491.0,518.0,hoard,1,0,0,NaN,NaN,...,1925.0,1925.0,One silver bowl with niello cross. Four stamps...,43.317,26.274,NaN,"// Gerasimov, T. - 1939 - Monetnite sakrovišt...",CHRE,2018-12-06 18:12:14.258392,2018-12-06 18:12:14.259510
123,CHRE-6343,COMMELLE-VERNAY,0.0,500.0,hoard,1,0,0,100.0,NaN,...,1824.0,1824.0,The hoard comprised approximately 100 Roman si...,45.998,4.059,NaN,TAF V. 3 (La Loire); See also Archives du Cabi...,CHRE,2018-12-06 18:12:14.258392,2018-12-06 18:12:14.259510
130,CHRE-6487,RUGGEL FL 1884,325.0,325.0,hoard,1,0,0,75.0,NaN,...,1884.0,1884.0,"25 coins found during peat digging, all from t...",47.242,9.525,NaN,"// Overbeck, Bernhard - 1973 - Geschichte des...",CHRE,2018-12-06 18:12:14.258392,2018-12-06 18:12:14.259510
134,CHRE-6514,BASEL BS - KLEINHÜNINGEN 1934 / TOMB 126,424.0,455.0,hoard,1,0,0,NaN,NaN,...,1934.0,1934.0,20 siliqua found in an early medieval burial (...,47.585,7.594,NaN,"// Giesler-Müller, Ulrike - 1992 - Das frühmi...",CHRE,2018-12-06 18:12:14.258392,2018-12-06 18:12:14.259510


In [0]:
coin_groups_fifth_filter.to_csv('CHRE-coin_groups.csv')
finds.to_csv('CHRE-coin_hoards.csv')
files.download('CHRE-coin_groups.csv')
files.download('CHRE-coin_hoards.csv')